In [1]:
# import sys
# import os
# sys.path.insert(1, os.getcwd())
# print(os.getcwd())
# from datetime import date
# from datetime import timedelta
# import logging
# import pandas as pd
# import uuid

# from data_retrieval.google_analytics_api_retrieval import GoogleAnalyticsApiRetrieval, GoogleAuthenticationMethod
# from helpers.file_helper import save_list_to_csv
# from helpers.settings_helper import get_google_analytics_view_id_from_settings, get_file_storage_root_folder_from_settings
# from helpers.metadata_helper import load_metadata, DataFrequency, DataModule, JobStatus, get_start_date
# from helpers.file_helper import create_directory_excluding_filename

CLIENT_SECRETS_FILE_PATH = './credentials/credentials.json'
TOKEN_FILE_PATH = './credentials/token.json'
# METADATA_FILE_PATH = "./metadata/metadata.json"
# SETTINGS_FILE_PATH = './settings/app_settings.json'


In [2]:
from data_retrieval.google_analytics_data import GoogleAnalyticsData
from datetime import date

ga_data = GoogleAnalyticsData(CLIENT_SECRETS_FILE_PATH, TOKEN_FILE_PATH)
run_id = ga_data.save_data(date(2022,1,1), date(2023,1,1))

/Users/pujanmaharjan/Projects/sacommunity/data-analytics


In [3]:
print('run id ', run_id)

run id  e3bea171-7973-4131-bccf-9c2a2ab84489


In [ ]:
# ga_data_log = logging.getLogger(__name__)
# view_id = get_google_analytics_view_id_from_settings()
# print('google analytics view id ', view_id)
# root_dir = get_file_storage_root_folder_from_settings()
# print('root_dir to store data is ', root_dir)
# run_id = str(uuid.uuid4())
# print('Run id ', run_id)

In [ ]:

# def save_df_to_csv(df: pd.DataFrame, root_dir: str, run_id: str, module: str):
#     file_name = f'{module}.csv'
#     file_path = os.path.join(root_dir, 'data', run_id, file_name)
#     create_directory_excluding_filename(file_path)
#     df.to_csv(file_path, index=False)


In [ ]:
# def get_unique_values(data, column):
#   num_unique_values = len(data[column].unique())
#   value_counts = data[column].value_counts()
#   print(f"Column: {column} has {num_unique_values} unique values\n")
#   print(value_counts)

In [ ]:
# def group_data(df : pd.DataFrame, select_columns: list[str], grp_columns: list[str]) -> pd.DataFrame:
#     df_s = df[select_columns]
#     df_grp = df_s.groupby(by=grp_columns, as_index=False).sum()
#     return df_grp

# def filter_data_by_dataset_id(df: pd.DataFrame, data_set_id: str):
#     return df[df['dataset_id'] == data_set_id]

In [ ]:
# from dtos.date_range_dto import DateRangeDto
# from data_retrieval.google_analytics_api_retrieval import PageDto
# ga = GoogleAnalyticsApiRetrieval(google_authentication_method = GoogleAuthenticationMethod.OAUTH,
#                                 oauth_credentials_filepath= CLIENT_SECRETS_FILE_PATH,
#                                 oauth_token_filepath=TOKEN_FILE_PATH,
#                                 view_id=view_id)

# date_range_dto = DateRangeDto(date(2022,1,1), date(2023,1,1))
# page_dto = PageDto(10000, None)

In [ ]:
# # 1, 2, 3. Data for device category, source medium and landing page
# data_df = ga.get_sessions_by_landing_page(date_range_dto, page_dto)

In [ ]:
# data_df.info()

In [ ]:
# # 1 device category
# device_category_df = group_data(data_df, ["dataset_id", "device_category","sessions"], ["dataset_id", "device_category"])
# save_df_to_csv(device_category_df, root_dir, run_id, 'device_category')
# device_category_df.head()

In [ ]:
# # 2 source medium
# source_medium_df = group_data(data_df, ["dataset_id", "source_medium","sessions"], ["dataset_id", "source_medium"])
# save_df_to_csv(source_medium_df, root_dir, run_id, 'source_medium')
# source_medium_df.head()

In [ ]:
# # 3 landing page
# landing_page_df = group_data(data_df, ["dataset_id", "landing_page","sessions"], ["dataset_id", "landing_page"])
# save_df_to_csv(landing_page_df, root_dir, run_id, 'landing_page')
# landing_page_df.head()

In [ ]:
# # 4 Age
# age_df = ga.get_sessions_by_age(date_range_dto, page_dto)
# save_df_to_csv(age_df, root_dir, run_id, 'age')

In [ ]:
# age_df.head()

In [ ]:
# filter_data_by_dataset_id(age_df, '06COP17')

In [ ]:
# # 5 gender
# gender_df = ga.get_sessions_by_gender(date_range_dto, page_dto)
# save_df_to_csv(gender_df, root_dir, run_id, 'gender')

In [ ]:
# gender_df

In [ ]:
# filter_data_by_dataset_id(gender_df, '06COP17')

In [ ]:
from helpers.cu_dataset_reader import read_cu_dataset_settings_pdf
pdf_file = './settings/CU datasets settings _ SAcommunity - Connecting Up Australia.pdf'
datasets_df = read_cu_dataset_settings_pdf(pdf_file, return_dataframe=True)


In [ ]:
datasets_df